In [2]:
from PCMCI_RNN.PCMCI_RNN import PCMCI_RNN

from PCMCI_RNN.data_generator import RegimeDataGenerator


In [3]:
import numpy as np
sign_12 = {
        'positive_sign': {
            'min_length': 70,
            'max_length': 100,
            'relationships': {
                1: {
                    0: {0: 0.2},
                    1: {1: 0.2, 0: 0.8}
                }
            }
        },
        'negative-sign': {
            'min_length': 70,
            'max_length': 100,
            'relationships': {
                1: {
                    0: {0: 0.2},
                    1: {1: 0.2, 0: -0.8}
                }
            }
        }
    }

T = 3000
generator = RegimeDataGenerator(
        regime_definitions=sign_12,
        n_vars=2,
        T=T,
        seed=43,
        noise_std=1.0,
        cycle_regimes=False,  # pick randomly, excluding last regime
    )

sign_12_data, assignments, number_of_switches= generator.generate_data()

print("Data shape:", sign_12_data.shape)
print("Assignments shape:", assignments.shape)
print("Unique regimes:", np.unique(assignments, return_counts=True))
print("Number of switches between regimes:", number_of_switches)

positive_mask = generator.create_mask_for_regime(assignments, "positive_sign")
print("Mask shape:", positive_mask.shape)

Data shape: (3000, 2)
Assignments shape: (3000,)
Unique regimes: (array(['negative-sign', 'positive_sign'], dtype='<U13'), array([1398, 1602]))
Number of switches between regimes: 34
Mask shape: (3000, 2)


In [15]:
pcmci = PCMCI_RNN(
        data=sign_12_data,
        lookback=10,
        rnn_type='LSTM',
        hidden_dim=10,
        significance_threshold=0.001,  # example threshold
        device='cpu',
        split_percentage=0.9,
    )

# -- PHASE 1: Discover parents (with batched training)
discovered_parents = pcmci.discover_parents(
    max_condition_set_size=2,
    epochs=5,
    batch_size=16
)

# Inspect adjacency after all iterations
final_adjacency = discovered_parents
print("\nFinal adjacency (parents) discovered:")
for y in range(sign_12_data.shape[1]):
    print(f"Variable {y} has parents: {sorted(list(final_adjacency[y]))}")

# We can also check adjacency and R^2 matrices:
for i, (A, R) in enumerate(zip(pcmci.adjacency_history, pcmci.r2_history)):
    print(f"\nIteration {i} adjacency matrix:\n{A}")
    print(f"Iteration {i} R^2 matrix:\n{R}")

# -- PHASE 2: MCI step
final_r2_dict = pcmci.mci_step(epochs=10, batch_size=16)
print("\nMCI step: final R^2 for each (Y, X) link among discovered parents:")
for (y, x), score in final_r2_dict.items():
    print(f"R^2(Y={y} | X={x}): {score:.3f}")


=== Condition set size = 0 ===

=== Condition set size = 1 ===
Skipping variable 0 at cond_size 1, not enough candidates

=== Condition set size = 2 ===
Skipping variable 0 at cond_size 2, not enough candidates
Skipping variable 1 at cond_size 2, not enough candidates

Final adjacency (parents) discovered:
Variable 0 has parents: [0]
Variable 1 has parents: [0, 1]

Iteration 0 adjacency matrix:
[[1 0]
 [1 1]]
Iteration 0 R^2 matrix:
[[0.06378996 0.        ]
 [0.00536275 0.1242882 ]]

Iteration 1 adjacency matrix:
[[1 0]
 [1 1]]
Iteration 1 R^2 matrix:
[[0.06378996 0.        ]
 [0.00799525 0.12636226]]

Iteration 2 adjacency matrix:
[[1 0]
 [1 1]]
Iteration 2 R^2 matrix:
[[0.06378996 0.        ]
 [0.00799525 0.12636226]]

MCI step: final R^2 for each (Y, X) link among discovered parents:
R^2(Y=0 | X=0): 0.060
R^2(Y=0 | X=1): 0.000
R^2(Y=1 | X=0): 0.005
R^2(Y=1 | X=1): 0.116


# Arrow direction

In [18]:
from PCMCI_RNN.data_generator import RegimeDataGenerator

arrow_direction = {
        '12-link': {
            'min_length': 70,
            'max_length': 100,
            'relationships': {
                1: {
                    0: {0: 0.2},
                    1: {1: 0.2, 0: 0.8}
                }
            }
        },
        '21-link': {
            'min_length': 70,
            'max_length': 100,
            'relationships': {
                1: {
                    0: {1: 0.8, 0: 0.2},
                    1: {1: 0.2}
                }
            }
        }
    }

T = 3000




    


In [21]:
generator = RegimeDataGenerator(
        regime_definitions=arrow_direction,
        n_vars=2,
        T=T,
        seed=43,
        noise_std=1.0,
        cycle_regimes=False,  # pick randomly, excluding last regime
    )

arrow_direction_data, assignments, number_of_switches= generator.generate_data()

print("Data shape:", sign_12_data.shape)
print("Assignments shape:", assignments.shape)
print("Unique regimes:", np.unique(assignments, return_counts=True))
print("Number of switches between regimes:", number_of_switches)

positive_mask = generator.create_mask_for_regime(assignments, "positive_sign")
print("Mask shape:", positive_mask.shape)

Data shape: (3000, 2)
Assignments shape: (3000,)
Unique regimes: (array(['12-link', '21-link'], dtype='<U7'), array([1398, 1602]))
Number of switches between regimes: 34
Mask shape: (3000, 2)


In [22]:
pcmci = PCMCI_RNN(
        data=arrow_direction_data,
        lookback=10,
        rnn_type='LSTM',
        hidden_dim=10,
        significance_threshold=0.001,  # example threshold
        device='cpu',
        split_percentage=0.9,
    )

# -- PHASE 1: Discover parents (with batched training)
discovered_parents = pcmci.discover_parents(
    max_condition_set_size=2,
    epochs=5,
    batch_size=16
)

# Inspect adjacency after all iterations
final_adjacency = discovered_parents
print("\nFinal adjacency (parents) discovered:")
for y in range(sign_12_data.shape[1]):
    print(f"Variable {y} has parents: {sorted(list(final_adjacency[y]))}")

# We can also check adjacency and R^2 matrices:
for i, (A, R) in enumerate(zip(pcmci.adjacency_history, pcmci.r2_history)):
    print(f"\nIteration {i} adjacency matrix:\n{A}")
    print(f"Iteration {i} R^2 matrix:\n{R}")

# -- PHASE 2: MCI step
final_r2_dict = pcmci.mci_step(epochs=10, batch_size=16)
print("\nMCI step: final R^2 for each (Y, X) link among discovered parents:")
for (y, x), score in final_r2_dict.items():
    print(f"R^2(Y={y} | X={x}): {score:.3f}")


=== Condition set size = 0 ===

=== Condition set size = 1 ===

=== Condition set size = 2 ===
Skipping variable 0 at cond_size 2, not enough candidates
Skipping variable 1 at cond_size 2, not enough candidates

Final adjacency (parents) discovered:
Variable 0 has parents: [0, 1]
Variable 1 has parents: [0, 1]

Iteration 0 adjacency matrix:
[[1 1]
 [1 1]]
Iteration 0 R^2 matrix:
[[0.05347419 0.19370085]
 [0.0558123  0.04865402]]

Iteration 1 adjacency matrix:
[[1 1]
 [1 1]]
Iteration 1 R^2 matrix:
[[0.03547686 0.18251771]
 [0.02602023 0.03987539]]

Iteration 2 adjacency matrix:
[[1 1]
 [1 1]]
Iteration 2 R^2 matrix:
[[0.03547686 0.18251771]
 [0.02602023 0.03987539]]

MCI step: final R^2 for each (Y, X) link among discovered parents:
R^2(Y=0 | X=0): 0.048
R^2(Y=0 | X=1): 0.199
R^2(Y=1 | X=0): 0.020
R^2(Y=1 | X=1): 0.017
